In [ ]:
!python train.py config/train_new_tinystories.py  --out_path="out-tinystories/model.eqx" --max_iters=250

In [5]:
!python train.py config/train_tinystories.py  --init_from="resume"  --out_path="out-tinystories/model2.eqx" --max_iters=250

Overriding config with config/train_tinystories.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_path = 'out-tinystories/model.eqx'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 20
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
tensorboard_log = True # override via command line if you like

dataset = 'tinystories'
gradient_accumulation_steps = 1
batch_size = 32
block_size = 100 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 16
n_head = 12
n_embd = 512
dropout = 0.0

learning_rate = 1e-4 # with baby networks can afford to go a bit higher
max_iters = 500
lr_decay_iters = 500 # make equal to max_iters usually
min_lr = 1e-5 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens

In [30]:
import jax
import equinox as eqx


emb = eqx.nn.Embedding(3, 4, key=jax.random.key(1))
x = jax.numpy.array([[1, 0, 0.0],[0,1,0]])

print(emb.weight.shape)
print(jax.numpy.matmul(x, emb.weight))
print(emb.weight[0])
print(emb.weight[1])
print(emb.weight[2])

(3, 4)
[[ 0.07545026 -1.0032071  -1.1431501  -0.14357094]
 [ 0.5904228  -1.4384178  -0.03499871  0.14507416]]
[ 0.07545026 -1.0032071  -1.1431501  -0.14357094]
[ 0.5904228  -1.4384178  -0.03499871  0.14507416]
[ 0.23655683  1.5627991  -1.1358407   0.9167496 ]
